<hr style="height:4px;border-width:10;color:blue;background-color:black">



<img src="img/logo.png" alt="Smiley face" width="100" height="100" align="left">


# Potential Talents


<hr style="height:4px;border-width:10;color:blue;background-color:black">

<br><br>

<hr style="height:2px;border-width:10;color:blue;background-color:black">

<img src="https://media1.giphy.com/media/KEAj1Hni8vLc0JcHpC/giphy.gif?cid=790b761137e5bc4519fcfbd3ec0229bda95e6655ace51675&rid=giphy.gif&ct=s" alt="Smiley face" width="60" height="60" align="left">

## INTRODUCTION
<hr style="height:2px;border-width:10;color:blue;background-color:black">


The goal of this project is to predict how fit the candidate is based on their available information. To achieve the desired goal, the data will go through four-stage:


- Explore and analyse the data.
- Cleaning
- Modeling.
- And finally, ranking.

So, let's first start with importing the required libraries, set the seeds value and read the data file.

In [1]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt


import nltk
import torch
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
import tensorflow as tf
import gensim.downloader as api
from nltk.corpus import stopwords
from pytorchtools import EarlyStopping
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.test.utils import get_tmpfile
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Set a seed value
seed_value= 12321 

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `torch` pseudo-random generator at a fixed value
torch.manual_seed(seed_value)

In [3]:
df = pd.read_csv('data/potential-talents - Aspiring human resources - seeking human resources.csv')

<hr style="height:2px;border-width:10;color:blue;background-color:black">

<img src="https://www.lingpad.com/wp-content/uploads/2021/09/Reports-Analysis.gif" alt="Smiley face" width="60" height="60" align="left">

## ANALYSIS
<hr style="height:2px;border-width:10;color:blue;background-color:black">


Let's explore the data...

In [4]:
df.head(2)

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,NaN


In [5]:
df.shape

(104, 5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


- The data shape is `(104, 5)`, and there are no missing values.
- The `fit` column is completely null, it's the targeted column, so we can drop it.

In [7]:
df.drop('fit', axis=1, inplace=True)

Let's see the number of unique values in each column

In [8]:
for col in df.columns:
    print(f'There is {df[col].nunique()} unique value in {col} column')

There is 104 unique value in id column
There is 52 unique value in job_title column
There is 41 unique value in location column
There is 33 unique value in connection column


There are `104` rows and there is only `52` unique job title, this data should be random so I believe there must be a higher number of job titles. Moreover, people tend to have close job titles on LinkedIn and similar websites but usually, they have different firm names, certifications... and so on.
Therefore, I will check if there are any duplicated values or not. 

In [9]:
print('The number of duplicated rows is:', df.duplicated().sum())
df[df.duplicated()].head(2)

The number of duplicated rows is: 0


,id,job_title,location,connection


This result may be affected by the `id` column, let's try to see if there are any duplicate values without the `id` column.

In [10]:
df_dup = df.drop(['id'], axis = 1)                    
print("Number of duplicate entries:", df_dup.duplicated().sum()) 

Number of duplicate entries: 51


There are `51` duplicated rows, and I believe this is due to an inaccurate web scraping program or something similar. Anyways, I will drop them.

In [11]:
# See duplicate entries
df[df_dup.duplicated(keep=False)]  

,id,job_title,location,connection
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85
1,2,Native English Teacher at EPIK (English Program in Korea),Kanada,500+
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+
...,...,...,...,...
60,61,HR Senior Specialist,San Francisco Bay Area,500+
61,62,Seeking Human Resources HRIS and Generalist Positions,Greater Philadelphia Area,500+
62,63,Student at Chapman University,"Lake Forest, California",2
63,64,"SVP, CHRO, Marketing & Communications, CSR Officer | ENGIE | Houston | The Woodlands | Energy | GPHR | SPHR","Houston, Texas Area",500+


I will drop duplicate rows and rebuild dataframe


In [12]:
df_dup = df_dup.drop_duplicates()                                    
df = pd.concat([df['id'], df_dup], axis = 1).dropna(axis = 0)   
print("Shape of non-duplicated dataframe:", df.shape)  

Shape of non-duplicated dataframe: (53, 4)


So, the new data shape is `(51, 3)`

In [13]:
df = df.reset_index(drop = True)

Let's take a closer look at the `job title` column...

In [14]:
df.job_title.value_counts()

Aspiring Human Resources Professional                                                                                    2
Student at Indiana University Kokomo - Business Management - \nRetail Manager at Delphi Hardware and Paint               1
Information Systems Specialist and Programmer with a love for data and organization.                                     1
Aspiring Human Resources Professional | Passionate about helping to create an inclusive and engaging work environment    1
Senior Human Resources Business Partner at Heil Environmental                                                            1
Seeking Human  Resources Opportunities. Open to travel and relocation.                                                   1
Human Resources Coordinator at InterContinental Buckhead Atlanta                                                         1
Human Resources Generalist at Loparex                                                                                    1
Business Intelli

As you can see, there are different firm names, shortcuts, and sometimes numbers like the year or a phone number, also some people write their region in their job title. Because the goal of this project is to find talented professionals for tech companies, I will search for all the firms in the job title and see their speciality, if it's in tech or not for the cleaning part later.

| Company    | Description     | Type       | 
|------------|-----------------| -----------| 
| `Ryan`     | Real estate| `Not Tech`  |
| `Epik`     | Web hosting company| `Tech`  |
| `Engie`    | Energy and petroleum| `Not Tech`  |
| `Buckhead Atlanta Hotels`      | Hotel and Tourism| `Not Tech`  |
| `Styczynski Lab`      | biotechnology| `Tech`  |
| `Loparex`      | Paper and film release liners| `Not Tech`  |
| `Delphi Hardware`      | Hardware store| `Not Tech`  |
| `JTI`      | Tobacco company| `Not Tech`  |
| `Travelers`      | Insurance company| `Tech`  |
| `Schwan's`      | Home Delivery company| `Not Tech`  |
| `GIS`      | Computer program | `Tech`  |
| `Luxottica`      | Eyewear company| `Not Tech`  |
| `Beneteau`      | Boating industry| `Not Tech`  |
| `ScottMadden`      | Management consulting firm| `Not Tech`  |
| `EY`      | Management consulting firm| `Not Tech`  |
| `Endemol`      | Media company| `Not Tech`  |
| `Nortia Staffing`      | HR company| `Not Tech`  |
| `Heil`      | Garbage trucks & trash trucks| `Not Tech`  |
| `Excellence Logging`      | Oilfield services company| `Not Tech`  |


And here is a list of all the abbreviation and their description.

| Abbreviation    | Description| 
|------------|-----------------| 
| `CHRO`     | Chief Human Resource Officer| 
| `SVP`     | Senior Vice President| 
| `CSR`     | Corporate Social Responsibility| 
| `GPHR`     | Global Professional in Human Resources| 
| `SPHR`     | strategic and policy-making certification| 
| `HRIS`     | Human resource management system| 

I want to see the most frequent words...

In [15]:
words_counts = Counter()
for i in range(len(df)):
    for word in df.job_title[i].split(" "):
        words_counts[word] += 1

print('Number of total words', len(words_counts))
words_counts.most_common()

Number of total words 221


[('Human', 34),
 ('Resources', 29),
 ('at', 22),
 ('and', 13),
 ('Aspiring', 11),
 ('|', 10),
 ('Professional', 7),
 ('in', 6),
 ('University', 6),
 ('Seeking', 6),
 ('Business', 5),
 ('Student', 5),
 ('Generalist', 5),
 ('Manager', 5),
 ('of', 4),
 ('Specialist', 4),
 ('&', 4),
 ('Management', 4),
 ('seeking', 4),
 ('', 4),
 ('an', 3),
 ('Director', 3),
 ('for', 3),
 ('College', 2),
 ('aspiring', 2),
 ('professional', 2),
 ('Coordinator', 2),
 ('HR', 2),
 ('Senior', 2),
 ('internship', 2),
 ('Resources,', 2),
 ('Staffing', 2),
 ('North', 2),
 ('a', 2),
 ('Resources.', 2),
 ('Major', 2),
 ('to', 2),
 ('Information', 2),
 ('Systems', 2),
 ('-', 2),
 ('Position', 2),
 ('2019', 1),
 ('C.T.', 1),
 ('Bauer', 1),
 ('Graduate', 1),
 ('(Magna', 1),
 ('Cum', 1),
 ('Laude)', 1),
 ('Native', 1),
 ('English', 1),
 ('Teacher', 1),
 ('EPIK', 1),
 ('(English', 1),
 ('Program', 1),
 ('Korea)', 1),
 ('People', 1),
 ('Development', 1),
 ('Ryan', 1),
 ('Advisory', 1),
 ('Board', 1),
 ('Member', 1),
 ('Ce

The top 6 words are `Human`, `Resources`, `at`, `and`, `Aspiring` and `Professional`. Words like **(and)** and **(at)** have no individual meaning, it's there for grammar and other reasons. Therefore, these words will be removed in the cleaning part, as well as for the **Special Characters** like `|` and `&`.

And before going to cleaning, I want to see the location column...

In [16]:
df.location.value_counts()

Houston, Texas Area                    4
Greater New York City Area             3
Raleigh-Durham, North Carolina Area    3
Amerika Birleşik Devletleri            2
Kanada                                 2
Greater Philadelphia Area              2
Austin, Texas Area                     2
Greater Atlanta Area                   2
New York, New York                     1
İzmir, Türkiye                         1
Jackson, Mississippi Area              1
Baltimore, Maryland                    1
Greater Chicago Area                   1
Atlanta, Georgia                       1
Chattanooga, Tennessee Area            1
San Jose, California                   1
Torrance, California                   1
Myrtle Beach, South Carolina Area      1
Greater Boston Area                    1
Las Vegas, Nevada Area                 1
Denton, Texas                          1
Gaithersburg, Maryland                 1
Katy, Texas                            1
Dallas/Fort Worth Area                 1
Los Angeles, Cal

I think the location column looks fine...

<hr style="height:2px;border-width:10;color:blue;background-color:black">

<img src="https://1.bp.blogspot.com/-jDgZMgM9d9w/VxJ0CrqbUoI/AAAAAAAAAkM/qYSSmn0WkLMPwTV84Lu7UKWTftUXXJd3gCKgB/s1600/robotin506.gif" alt="Smiley face" width="60" height="60" align="left">

## CLEANING
<hr style="height:2px;border-width:10;color:blue;background-color:black">


In this part, I will do several things:
-  I will replace the firm names with `tech` or `not tech` depending on the firm specialities.
- Remove special characters and punctuation.
- Abbreviations replacement with their description.
- Because the Keywords are: **Aspiring human resources** or **seeking human resources**, I will replace all of the `HR` to `human resources`.
- All the text is written in lower case.
- Remove stopwords and apply lemmatization.

Let's clean the text by removing special characters and punctuation.

In [17]:
df = df.replace({'location' : { "[\'!#)$%&(*+-./:;<=>?@[\]^_`{|}~\n]" : " "}}, regex=True)
df = df.replace({'connection' : { "[\'!#)$%&(*+-./:;<=>?@[\]^_`{|}~\n]" : " "}}, regex=True)

And now, let's convert all the text in lower case...

In [18]:
df['job_title'] = df['job_title'].str.lower()
df['location'] = df['location'].str.lower()
df.head(2)

,id,job_title,location,connection
0,1,2019 c.t. bauer college of business graduate (magna cum laude) and aspiring human resources professional,houston texas,85
1,2,native english teacher at epik (english program in korea),kanada,500


Let's remove the `stopwords` and apply `lemmatization` by using **cleaning** function.

In [3]:
def cleaning(df, col):
    
    stop_words = set(stopwords.words('english'))
    for i in range (len(df)):
        word_tokens = word_tokenize(df[col][i])
        tokens_without_sw = [w for w in word_tokens if w not in stop_words]
        lemmatized_sentence = []
        for word in tokens_without_sw:
            lemmatized_sentence.append(WordNetLemmatizer().lemmatize(word))
        df[col][i] = TreebankWordDetokenizer().detokenize(lemmatized_sentence) 


In [20]:
print('Job title before removing stopwprds:\n', df.job_title.head(1))
print("-" * 100)

# Remove stop words and lemmatize words in cleaned job title
cleaning(df, 'job_title')

print('Job title after removing stopwprds:\n', df.job_title.head(1))
print("-" * 100)
df.head(2)

Job title before removing stopwprds:
 0    2019 c.t. bauer college of business graduate (magna cum laude) and aspiring human resources professional
Name: job_title, dtype: object
----------------------------------------------------------------------------------------------------
Job title after removing stopwprds:
 0    2019 c.t . bauer college business graduate (magna cum laude) aspiring human resource professional
Name: job_title, dtype: object
----------------------------------------------------------------------------------------------------


,id,job_title,location,connection
0,1,2019 c.t . bauer college business graduate (magna cum laude) aspiring human resource professional,houston texas,85
1,2,native english teacher epik (english program korea ),kanada,500


So by removing **stopwords**, the words like `of` and `and` have been removed. And this step can help to improve the model by reducing the noise.

And for the final step, let's replace the abbreviations with their description.

In [21]:
df = df.replace({'job_title' : { 'chro' : 'chief human resources officer', 'svp' : 'senior vice president'
        ,'gphr' : 'global professional in human resources','hris' : 'human resources management system'
        , 'csr' : 'corporate social responsibility', 'sphr' : 'strategic and policy-making certification'
        , 'hr' : 'human resources', "[\'!#)$%&(*+-./:;<=>?@[\]^_`{|}~\n]" : "", r'[0-9]' : '', 'epik' : 'tech'
        , 'styczynski lab' : 'tech', 'gi' : 'tech', "schwan's" : 'tech', 'ryan' : 'not tech'
        , 'engine' : 'not tech', 'buckhead atlanta' : 'not tech', 'loparex' : 'not tech', 'delphi Hardware' : 'not tech'
        , "jti" : 'not tech', 'traveler' : 'not tech', 'luxottica' : 'not tech'
        , 'beneteau' : 'not tech', 'scottMadden' : 'not tech', 'ey' : 'not tech', 'endemol' : 'not tech'
        , 'nortia staffing' : 'not tech', 'heil environmental' : 'not tech', 'excellence logging' : 'not tech'}}
        , regex=True)

<hr style="height:2px;border-width:10;color:blue;background-color:black">

<img src="https://cdnb.artstation.com/p/assets/images/images/019/834/893/original/viktor-stambert-character-veo-run-animation-1.gif?1565197751" alt="Smiley face" width="60" height="60" align="left">

##   Word Embedding Techniques
<hr style="height:2px;border-width:10;color:blue;background-color:black">


I will use 4 different strategies to find the similarities between the targeted sentences and each job title as follows:

1. **`TF-IDF`**: a statistical measure used to determine the mathematical significance of words in documents.

2. **`GloVe `**: focuses on words co-occurrences over the whole corpus. Its embeddings relate to the probabilities that two words appear together.

3. **`Word2Vec`** ==> `Google News` model: takes texts as training data for a neural network. The resulting embedding captures whether words appear in similar contexts.

4. **`FastText`**: improves on Word2Vec by taking word parts into account, too. This trick enables training of embeddings on smaller datasets and generalization to unknown words.

3. **`BERT`**: use of Transformer, an attention mechanism that learns contextual relations between words (or sub-words) in a text.

### 1. **TF-IDF**

In [22]:
# Convert job_title column into a list
job_title_list = list(df['job_title'])                    

# Vectorize job_title_list
vectorizer = TfidfVectorizer().fit(job_title_list)                    
X = vectorizer.transform(job_title_list)   

# Get feature names in all the documents
feature_names = vectorizer.get_feature_names()                  
print("Number of unique features: ", len(feature_names)) 
print("First 5 features: ", feature_names[:5]) 

# Convert job titles into arrays
tfidf_vector = X.toarray()                                  
print("Shape of Tfidf vector: ", tfidf_vector.shape)  

Number of unique features:  167
First 5 features:  ['administration', 'administrative', 'admission', 'advisory', 'always']
Shape of Tfidf vector:  (53, 167)


What's happened in the previous cell is we converted all the unique words to columns and fill the rows by the TF-IDF value. And now, we can calculate the relation between the targeted sentence and each job title...

#### Calculate the similarity

To calculate the similarity, I will use the `similarity` function.

In [23]:
def similarity(phrase, column_name):
    # Convert search phrase into a vector
    X = vectorizer.transform(phrase)                    
    X_vector = X.toarray()
    print("Shape of search phrase vector:", X_vector.shape)
    
    # Calcualte Tfidf cosine similarity and add it to pt dataframe
    sim_score_list = []
    for x in range (len(df)):
        sim_score_list.append(1 - cosine(tfidf_vector[x], X_vector))

    df[column_name] = sim_score_list


In [4]:
# Searched phrase
phrases = pd.DataFrame({'phrase' : ['aspiring human resources']})
cleaning(phrases, 'phrase')

In [25]:
similarity([phrases.phrase[0]], 'TF-IDF_fitt_score')

Shape of search phrase vector: (1, 167)


Let's see the top and lowest matching `title job` rows with the phrase **(aspiring human resources)**.

In [26]:
df.sort_values(by ='TF-IDF_fitt_score', ascending = False).head()

,id,job_title,location,connection,TF-IDF_fitt_score
2,3,aspiring human resource professional,raleigh durham north carolina area,44,0.781184
45,97,aspiring human resource professional,kokomo indiana area,71,0.781184
5,6,aspiring human resource specialist,greater new york city area,1,0.679890
21,73,aspiring human resource manager seeking internship human resource,houston texas area,7,0.618600
12,27,aspiring human resource management student seeking internship,houston texas area,500,0.442293


In [27]:
df.sort_values(by ='TF-IDF_fitt_score', ascending = False).tail()

,id,job_title,location,connection,TF-IDF_fitt_score
34,86,information system specialist programmer love data organization,gaithersburg maryland,4,0.0
33,85,rrp brand portfolio executive not tech japan tobacco international,greater philadelphia area,500,0.0
28,80,junior me entechneer information system,myrtle beach south carolina area,52,0.0
1,2,native english teacher tech english program korea,kanada,500,0.0
52,104,director administration excellence logtechng,katy texas,500,0.0


- As we can see the top rows have the same phrase in their job title and all the lowest ones have no relation with the targeted phrase at all.
- Other words existing alongside the targeted phrase seems to reduce the fit score but this will be different after ranking or using different ways to find the similarities.

### 2. **GloVe**

In [28]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

In [29]:
from gensim.test.utils import get_tmpfile

# Create temp file and save converted embedding into it
target_file = get_tmpfile('word2vec.6B.50d.txt')
glove2word2vec('glove.6B.50d.txt', target_file) 

# Load the converted embedding into memory
glove_model = KeyedVectors.load_word2vec_format(target_file)

# Save as binary data
glove_model.save_word2vec_format('word2vec.6B.50d.bin.gz', binary=True)

I will `doc_token_vectors` function to vectorize a sentence's words, and I will use the `model_fitt_score` function to find  the fitt score with `GloVe` model.

In [9]:
def doc_token_vectors(sentence, model, sentence_vector_list, vector_dimensions):
    word_tokens = word_tokenize(sentence)
    filtered_words = [w for w in word_tokens if w in model]
    for j in range(len(word_tokens)):
        if word_tokens[j] in filtered_words:
            token_vector = model[word_tokens[j]]
        else:
            token_vector = np.zeros(vector_dimensions)
        sentence_vector_list.append(token_vector)
    
    return sentence_vector_list

In [10]:
def model_fitt_score(df, col, model, vector_dimensions, phrase, fitt_col_name):
    # Vectorize job title using the model
    model_vectors = []
    for i in range(len(df)):
        model_sentence_vector = []
        doc_token_vectors(df[col][i], model, model_sentence_vector, vector_dimensions)
        model_vectors.append(model_sentence_vector)

    print('model vectors shape', np.shape(model_vectors))
    # Vectorize searched phrase using the model
    model_search_phrase_vector = []
    doc_token_vectors(phrase, model, model_search_phrase_vector, vector_dimensions)
    print('model search phrase vector shape', np.shape(model_search_phrase_vector))

    # Calculate cosine similarity between searched phrase and job title
    model_similarity =[]
    for i in range(len(df)):
        sim_score = 1 - cosine(np.mean(model_vectors[i], axis = 0), np.mean(model_search_phrase_vector, axis =0))
        model_similarity.append(sim_score)

    # Add model similarity score to the pt dataframe
    df[fitt_col_name] = model_similarity
    # return df


In [32]:
model_fitt_score(df, 'job_title', glove_model, 50, phrases.phrase[0], 'GloVe_fitt_score')

model vectors shape (53,)
model search phrase vector shape (3, 50)


In [33]:
df.sort_values(by ='GloVe_fitt_score', ascending = False).head()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score
5,6,aspiring human resource specialist,greater new york city area,1,0.679890,0.969614
2,3,aspiring human resource professional,raleigh durham north carolina area,44,0.781184,0.958512
45,97,aspiring human resource professional,kokomo indiana area,71,0.781184,0.958512
21,73,aspiring human resource manager seeking internship human resource,houston texas area,7,0.618600,0.934672
22,74,human resource professional,greater boston area,16,0.421758,0.933199


In [34]:
df.sort_values(by ='GloVe_fitt_score', ascending = False).tail()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score
9,11,student chapman university,lake forest california,2,0.0,0.519102
35,87,bachelor science biology victoria university wellington,baltimore maryland,40,0.0,0.518551
44,96,student indiana university kokomo business management retail manager delphi hardware paint,lafayette indiana,19,0.0,0.517594
43,95,student westfield state university,bridgewater massachusetts,57,0.0,0.511535
4,5,advisory board member celal bayar university,i̇zmir türkiye,500,0.0,0.450873


As in the TF-IDF, the top rows have the same phrase in their job title. The big difference between the TF-IDF and GloVe model result is the last rows score. In the TF-IDF, it's scoring the unrelated titles with **0** but the GloVe model gives them a high score, which is about **50%** of matching!!

### 3. **Word2vec**

In [35]:
GoogleNews_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [36]:
model_fitt_score(df, 'job_title', GoogleNews_model, 300, phrases.phrase[0], 'GoogleNews_fitt_score')

model vectors shape (53,)
model search phrase vector shape (3, 300)


In [37]:
df.sort_values(by ='GoogleNews_fitt_score', ascending = False).head()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score
45,97,aspiring human resource professional,kokomo indiana area,71,0.781184,0.958512,0.950395
2,3,aspiring human resource professional,raleigh durham north carolina area,44,0.781184,0.958512,0.950395
5,6,aspiring human resource specialist,greater new york city area,1,0.679890,0.969614,0.912262
21,73,aspiring human resource manager seeking internship human resource,houston texas area,7,0.618600,0.934672,0.875945
22,74,human resource professional,greater boston area,16,0.421758,0.933199,0.874494


In [38]:
df.sort_values(by ='GoogleNews_fitt_score', ascending = False).tail()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score
46,98,student,houston texas area,4,0.0,0.626729,0.246259
9,11,student chapman university,lake forest california,2,0.0,0.519102,0.244274
51,103,always set success,greater los angeles area,500,0.0,0.631108,0.213981
4,5,advisory board member celal bayar university,i̇zmir türkiye,500,0.0,0.450873,0.206563
52,104,director administration excellence logtechng,katy texas,500,0.0,0.665444,0.191680


The top scores for the word2vec model are close to the GloVe model, but the last five rows get a lower score which is good.

I'm using Colab to run this project, so due to the ram issue, I will save my work and start from this new point.

In [39]:
df.to_csv('df_stop_point_1.csv', index=False)

In [5]:
df = pd.read_csv('df_stop_point_1.csv')
df.head(1)

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score
0,1,ct bauer college business graduate magna cum laude aspiring human resource professional,houston texas,85,0.254004,0.627218,0.592438


### 4. **Fast text model**

In [6]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
# !gunzip /content/cc.fa.300.bin.gz
# !pip install fasttext

In [7]:
import fasttext 
FT_model = fasttext.load_model("/content/cc.fa.300.bin")

In [11]:
model_fitt_score(df, 'job_title', FT_model, 300, phrases.phrase[0], 'FT_model_fitt_score')


model vectors shape (53,)
model search phrase vector shape (3, 300)


In [12]:
df.sort_values(by ='FT_model_fitt_score', ascending = False).head()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score,FT_model_fitt_score
45,97,aspiring human resource professional,kokomo indiana area,71,0.781184,0.958512,0.950395,0.974163
2,3,aspiring human resource professional,raleigh durham north carolina area,44,0.781184,0.958512,0.950395,0.974163
22,74,human resource professional,greater boston area,16,0.421758,0.933199,0.874494,0.974163
5,6,aspiring human resource specialist,greater new york city area,1,0.679890,0.969614,0.912262,0.969252
21,73,aspiring human resource manager seeking internship human resource,houston texas area,7,0.618600,0.934672,0.875945,0.956482


In [13]:
df.sort_values(by ='FT_model_fitt_score', ascending = False).tail()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score,FT_model_fitt_score
4,5,advisory board member celal bayar university,i̇zmir türkiye,500,0.0,0.450873,0.206563,0.423610
51,103,always set success,greater los angeles area,500,0.0,0.631108,0.213981,0.396009
46,98,student,houston texas area,4,0.0,0.626729,0.246259,0.378976
52,104,director administration excellence logtechng,katy texas,500,0.0,0.665444,0.191680,0.368831
28,80,junior me entechneer information system,myrtle beach south carolina area,52,0.0,0.686935,0.356155,0.305511


The Fast Text model gives the top rows a high score but at the same time, it gives the last rows a higher score also.

### 5. BERT

In [15]:
# pip install sentence_transformers

In [16]:
from sentence_transformers import SentenceTransformer
# Build BERT_base model
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# Convert job titles into BERT embedded vectors
bert_job_title_embeddings = bert_model.encode(list(df.job_title))
bert_job_title_embeddings.shape

(53, 768)

In [18]:
# Convert search phrase into a BERT embedded vector
bert_search_phrase_embedding = bert_model.encode(phrases.phrase[0])
bert_search_phrase_embedding.shape

(768,)

In [19]:
# Calculate cosine similarity between job title and search phrase vectors
bert_cosine_similarity = []
for i in range(len(df)):
    cos_sim = 1 - cosine(bert_job_title_embeddings[i], bert_search_phrase_embedding)
    bert_cosine_similarity.append(cos_sim)
    
# Add BERT_cosine_similarity column in the pt dataframe
df['BERT_model_fitt_score'] = bert_cosine_similarity

In [20]:
df.sort_values(by ='BERT_model_fitt_score', ascending = False).head()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score,FT_model_fitt_score,BERT_model_fitt_score
5,6,aspiring human resource specialist,greater new york city area,1,0.679890,0.969614,0.912262,0.969252,0.955137
2,3,aspiring human resource professional,raleigh durham north carolina area,44,0.781184,0.958512,0.950395,0.974163,0.948828
45,97,aspiring human resource professional,kokomo indiana area,71,0.781184,0.958512,0.950395,0.974163,0.948828
30,82,aspiring human resource professional energetic teamfocused leader,austin texas area,174,0.379605,0.895589,0.827266,0.850075,0.867910
47,99,seeking human resource position,las vegas nevada area,48,0.289466,0.869983,0.728513,0.885787,0.849293


In [21]:
df.sort_values(by ='BERT_model_fitt_score', ascending = False).tail()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score,FT_model_fitt_score,BERT_model_fitt_score
44,96,student indiana university kokomo business management retail manager delphi hardware paint,lafayette indiana,19,0.0,0.517594,0.314945,0.537046,0.230208
50,102,business intelligence analytics not tech,greater new york city area,49,0.0,0.662951,0.344934,0.509455,0.198931
41,93,admission representative community medical center long beach,long beach california,9,0.0,0.664062,0.262623,0.501026,0.181031
33,85,rrp brand portfolio executive not tech japan tobacco international,greater philadelphia area,500,0.0,0.579212,0.271064,0.535838,0.147233
35,87,bachelor science biology victoria university wellington,baltimore maryland,40,0.0,0.518551,0.277453,0.501167,0.136410


    Among all of the models, the BERT model gives us a high score for the top rows and a low one for unrelated titles.

<hr style="height:2px;border-width:10;color:blue;background-color:black">

<img src="https://tonettedelaluna.files.wordpress.com/2021/04/source.gif" alt="Smiley face" width="60" height="60" align="left">

##   Learning to Rank (LTR)

<hr style="height:2px;border-width:10;color:blue;background-color:black">


In [22]:
df

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score,FT_model_fitt_score,BERT_model_fitt_score
0,1,ct bauer college business graduate magna cum laude aspiring human resource professional,houston texas,85,0.254004,0.627218,0.592438,0.593629,0.557126
1,2,native english teacher tech english program korea,kanada,500,0.000000,0.650306,0.283308,0.472039,0.408490
2,3,aspiring human resource professional,raleigh durham north carolina area,44,0.781184,0.958512,0.950395,0.974163,0.948828
3,4,people development coordinator not tech,denton texas,500,0.000000,0.734886,0.358650,0.513124,0.318839
4,5,advisory board member celal bayar university,i̇zmir türkiye,500,0.000000,0.450873,0.206563,0.423610,0.461810
5,6,aspiring human resource specialist,greater new york city area,1,0.679890,0.969614,0.912262,0.969252,0.955137
6,7,student humber college aspiring human resource generalist,kanada,61,0.395109,0.835481,0.829068,0.809919,0.712708
7,8,human resources senior specialist,san francisco bay area,500,0.104209,0.849321,0.597517,0.856602,0.703114
8,10,seeking human resource human resources management system generalist position,greater philadelphia area,500,0.237450,0.871824,0.729699,0.905917,0.721451
9,11,student chapman university,lake forest california,2,0.000000,0.519102,0.244274,0.441249,0.426179


In [23]:
star_candidate = input("Do you want to star any candidates? Enter 'Yes' or 'No': ")

starred = []
if star_candidate.lower() == 'yes':
    starred = [int(item) for item in input("Enter ids of candidates you want to star (separated by space) : ").split()]

Do you want to star any candidates? Enter 'Yes' or 'No': yes
Enter ids of candidates you want to star (separated by space) : 8 12 84


In [24]:
df['starred_score'] = df['BERT_model_fitt_score']
for id_num in starred:
    df.loc[df['id'] == id_num, 'starred_score'] = 1
df.head()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score,FT_model_fitt_score,BERT_model_fitt_score,starred_score
0,1,ct bauer college business graduate magna cum laude aspiring human resource professional,houston texas,85,0.254004,0.627218,0.592438,0.593629,0.557126,0.557126
1,2,native english teacher tech english program korea,kanada,500,0.000000,0.650306,0.283308,0.472039,0.408490,0.408490
2,3,aspiring human resource professional,raleigh durham north carolina area,44,0.781184,0.958512,0.950395,0.974163,0.948828,0.948828
3,4,people development coordinator not tech,denton texas,500,0.000000,0.734886,0.358650,0.513124,0.318839,0.318839
4,5,advisory board member celal bayar university,i̇zmir türkiye,500,0.000000,0.450873,0.206563,0.423610,0.461810,0.461810


In [25]:
class RankNet(nn.Module):
    
    def __init__(self, num_feature):
        super(RankNet, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(num_feature, 512),         
            nn.Dropout(0.5),                     
            nn.LeakyReLU(0.2, inplace=True),     
            nn.Linear(512, 256),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid())
        
        self.output_sig = nn.Sigmoid()

    def forward(self, input_1, input_2):
        s1 = self.model(input_1)
        s2 = self.model(input_2)
        out = self.output_sig(s1-s2)
        return out
    
    def predict(self, input_):
        s = self.model(input_)
        return s

In [26]:
random_row_1 = df.sample(n = 100, replace = True)
random_row_2 = df.sample(n = 100, replace = True)
job_title_list_ranknet1 = list(random_row_1['job_title'])
job_title_list_ranknet2 = list(random_row_2['job_title'])
doc_1 = bert_model.encode(job_title_list_ranknet1)
doc_2 = bert_model.encode(job_title_list_ranknet2)
doc_1 = torch.from_numpy(doc_1).float()
doc_2 = torch.from_numpy(doc_2).float()

In [27]:
y_1 = list(random_row_1['starred_score'])
y_2 = list(random_row_2['starred_score'])
y = torch.tensor([1.0 if y1_i>y2_i else 0.5 if y1_i==y2_i else 0.0 for y1_i, y2_i in zip(y_1, y_2)]).float()

y = y.unsqueeze(1)


In [37]:
def train_model(optim, lr_list, epoch, patience):
    # to track the training loss as the model trains
    losses = []

    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)


    model = RankNet(num_feature = 768)
    loss_fun = torch.nn.BCELoss()

    for lr in lr_list:
        if optim == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = 0.9)
        elif optim == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr = lr)
        print('lr: ', lr, 'optimizer: ', optim)
        epoch = epoch
        train_losses = []
        valid_losses = []

        for i in range(1, epoch+1):
            model.zero_grad()
            y_pred = model(doc_1, doc_2)
            loss = loss_fun(y_pred, y)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

            if i % 100 == 0:
                print('Epoch{}, loss : {}'.format(i, loss.item()))
        print('=' * 100)


        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    return  model, losses


In [32]:
model, train_loss= train_model(optim='SGD', lr_list = [0.1, 0.01, 0.001, 0.0001, 0.00001], epoch=1000, patience=20)

lr:  0.1 optimizer:  SGD
Epoch100, loss : 0.5028175115585327
Epoch200, loss : 0.49125778675079346
Epoch300, loss : 0.49507319927215576
Epoch400, loss : 0.4987936317920685
Epoch500, loss : 0.48969849944114685
Epoch600, loss : 0.4928869307041168
Epoch700, loss : 0.494904100894928
Epoch800, loss : 0.49159881472587585
Epoch900, loss : 0.49898940324783325
Epoch1000, loss : 0.49697792530059814
Validation loss decreased (inf --> 0.496978).  Saving model ...
lr:  0.01 optimizer:  SGD
Epoch100, loss : 0.49201500415802
Epoch200, loss : 0.49372780323028564
Epoch300, loss : 0.48953932523727417
Epoch400, loss : 0.4983752369880676
Epoch500, loss : 0.4908524751663208
Epoch600, loss : 0.49481120705604553
Epoch700, loss : 0.4894487261772156
Epoch800, loss : 0.48937591910362244
Epoch900, loss : 0.48875683546066284
Epoch1000, loss : 0.4904109835624695
Validation loss decreased (0.496978 --> 0.490411).  Saving model ...
lr:  0.001 optimizer:  SGD
Epoch100, loss : 0.4924342632293701
Epoch200, loss : 0.4886

In [38]:
model, train_loss= train_model(optim='SGD', lr_list = [0.01], epoch=2000, patience=20)

lr:  0.01 optimizer:  SGD
Epoch100, loss : 0.513862669467926
Epoch200, loss : 0.49813953042030334
Epoch300, loss : 0.49450263381004333
Epoch400, loss : 0.4950868487358093
Epoch500, loss : 0.49618470668792725
Epoch600, loss : 0.4970145523548126
Epoch700, loss : 0.4954339861869812
Epoch800, loss : 0.4930400550365448
Epoch900, loss : 0.4957929849624634
Epoch1000, loss : 0.49455055594444275
Epoch1100, loss : 0.49688220024108887
Epoch1200, loss : 0.491041898727417
Epoch1300, loss : 0.4927457869052887
Epoch1400, loss : 0.49047374725341797
Epoch1500, loss : 0.4940037429332733
Epoch1600, loss : 0.496761679649353
Epoch1700, loss : 0.4946865141391754
Epoch1800, loss : 0.49511319398880005
Epoch1900, loss : 0.48914313316345215
Epoch2000, loss : 0.49186477065086365
Validation loss decreased (inf --> 0.491865).  Saving model ...


In [34]:
model, train_loss = train_model(optim='Adam', lr_list = [0.1, 0.01, 0.001, 0.0001, 0.00001], epoch=1000, patience=20)

lr:  0.1 optimizer:  Adam
Epoch100, loss : 0.6931471228599548
Epoch200, loss : 0.6931471228599548
Epoch300, loss : 0.6931471228599548
Epoch400, loss : 0.6931471228599548
Epoch500, loss : 0.6931471228599548
Epoch600, loss : 0.6931471228599548
Epoch700, loss : 0.6931471228599548
Epoch800, loss : 0.6931471228599548
Epoch900, loss : 0.6931471228599548
Epoch1000, loss : 0.6931471228599548
Validation loss decreased (inf --> 0.693147).  Saving model ...
lr:  0.01 optimizer:  Adam
Epoch100, loss : 0.6931471228599548
Epoch200, loss : 0.6931471228599548
Epoch300, loss : 0.6931471228599548
Epoch400, loss : 0.6931471228599548
Epoch500, loss : 0.6931471228599548
Epoch600, loss : 0.6931471228599548
Epoch700, loss : 0.6931471228599548
Epoch800, loss : 0.6931471228599548
Epoch900, loss : 0.6931471228599548
Epoch1000, loss : 0.6931471228599548
Validation loss decreased (0.693147 --> 0.693147).  Saving model ...
lr:  0.001 optimizer:  Adam
Epoch100, loss : 0.6931471228599548
Epoch200, loss : 0.693147122

In [35]:
class RankNet(nn.Module):
    
    def __init__(self, num_feature):
        super(RankNet, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(num_feature, num_feature * 2),        
            nn.Dropout(0.2),                     
            nn.LeakyReLU(0.2, inplace=True),     
            nn.Linear(num_feature * 2, num_feature * 4),
            nn.Dropout(0.2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(num_feature * 4, 1),
            nn.Sigmoid())
        self.output_sig = nn.Sigmoid()

    def forward(self, input_1, input_2):
        s1 = self.model(input_1)
        s2 = self.model(input_2)
        out = self.output_sig(s1-s2)
        return out
    
    def predict(self, input_):
        s = self.model(input_)
        return s

In [36]:
model, train_loss= train_model(optim='SGD', lr_list = [0.01], epoch=2000, patience=20)

lr:  0.01 optimizer:  SGD
Epoch100, loss : 0.5151602029800415
Epoch200, loss : 0.49959877133369446
Epoch300, loss : 0.495458722114563
Epoch400, loss : 0.4952317178249359
Epoch500, loss : 0.49099960923194885
Epoch600, loss : 0.49100062251091003
Epoch700, loss : 0.49090901017189026
Epoch800, loss : 0.49341487884521484
Epoch900, loss : 0.49321234226226807
Epoch1000, loss : 0.49638691544532776
Epoch1100, loss : 0.49171218276023865
Epoch1200, loss : 0.4918365776538849
Epoch1300, loss : 0.4927673637866974
Epoch1400, loss : 0.4938880503177643
Epoch1500, loss : 0.49174919724464417
Epoch1600, loss : 0.4929833710193634
Epoch1700, loss : 0.491707444190979
Epoch1800, loss : 0.4915185570716858
Epoch1900, loss : 0.49521133303642273
Epoch2000, loss : 0.49224430322647095
Validation loss decreased (inf --> 0.492244).  Saving model ...


In [39]:
pred_score = []
for i in range(len(df)):
    embedding = bert_model.encode([df['job_title'][i]])
    embedding_tensor = torch.from_numpy(embedding).float()
    pred = round(model.predict(embedding_tensor).detach().numpy().sum(),2)
    pred_score.append(pred)

df['RankNet_score'] = pred_score
df.sort_values(by ='RankNet_score', ascending = False).head()

,id,job_title,location,connection,TF-IDF_fitt_score,GloVe_fitt_score,GoogleNews_fitt_score,FT_model_fitt_score,BERT_model_fitt_score,starred_score,RankNet_score
0,1,ct bauer college business graduate magna cum laude aspiring human resource professional,houston texas,85,0.254004,0.627218,0.592438,0.593629,0.557126,0.557126,1.0
15,67,human resource staffing recruiting professional,jackson mississippi area,500,0.198971,0.848083,0.706574,0.955237,0.714154,0.714154,1.0
48,100,aspiring human resource manager graduating may seeking entrylevel human resource position st louis,cape girardeau missouri,103,0.378556,0.797950,0.791290,0.627699,0.593210,0.593210,1.0
47,99,seeking human resource position,las vegas nevada area,48,0.289466,0.869983,0.728513,0.885787,0.849293,0.849293,1.0
45,97,aspiring human resource professional,kokomo indiana area,71,0.781184,0.958512,0.950395,0.974163,0.948828,0.948828,1.0


<hr style="height:2px;border-width:10;color:blue;background-color:black">

<img src="https://media1.giphy.com/media/HL5fq93TXz4sW3u7EL/giphy.gif?cid=790b7611dead5ce6a99ec8a7ace650b2a30c46418e5f871c&rid=giphy.gif&ct=s" alt="Smiley face" width="60" height="60" align="left">

##   CONCLUSION
<hr style="height:2px;border-width:10;color:blue;background-color:black">


BERT model was one of the best models to find the similarity between our data and the targeted phrase, and that's why I used its score to feed the model the first time. As for the Ranking model, I run the RankNet model on the data and our best loos score is 48%. I believe the data size play a major role to get this result, even TF-IDF gives us a more reliable result in my opinion than the RankNet model.